# Imports

In [ ]:
import sys
from keras.models import load_model
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from skimage.io import imread
from sklearn.model_selection import train_test_split

import os
from os import listdir, makedirs
from os.path import join, exists, expanduser
import numpy as np
import pandas as pd
import cv2
from skimage.io import imread

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf
import efficientnet.keras as efn

# Variables

In [ ]:
# Dimensiones imagenes
img_width, img_height = 224, 224

#Rutas Train y test
train_data_dir = 'C:/Bootcamp/repositorio/MachineLearningProject/data/EVs/models/'
TEST_PATH = 'C:/Bootcamp/repositorio/MachineLearningProject/data/EVs/test/'

#varios
batch_size = 32


# ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Modelo

In [ ]:
model = efn.EfficientNetB7(input_shape=(img_width, img_height, 3),
                           weights='imagenet',
                           include_top = False, 
                           classes=11)

# Layers

In [ ]:
for layer in model.layers:
    layer.trainable = False
    
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="sigmoid")(x)
predictions = Dense(11, activation="softmax")(x)
model_final = Model(model.input, predictions)

# Compile

In [ ]:
model_final.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# Entrenamiento

In [ ]:
history_nasa = model_final.fit(
    train_generator,
    epochs=10, 
    shuffle = True, 
    verbose = 1,
    validation_data=validation_generator)

# Salvamos el modelo

In [ ]:
model_final.save('EfficientNetB7.h5')